In [13]:
import pandas as pd

import re
import string

import yaml

from nltk.stem.porter import PorterStemmer

In [14]:
with open("../../config.yaml", "r") as stream:
    try:
        configs = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [15]:
PATH_TO_TRAIN_DS_RAW = str(configs['ROOT_DIR'] + configs['PATH_TO_TRAIN_DS_RAW'])
PATH_TO_TEST_DS_RAW  = str(configs['ROOT_DIR'] + configs['PATH_TO_TEST_DS_RAW'])

PATH_TO_TRAIN_DS_PROCESSED = str(configs['ROOT_DIR'] + configs['PATH_TO_TRAIN_DS_PROCESSED'])
PATH_TO_TEST_DS_PROCESSED  = str(configs['ROOT_DIR'] + configs['PATH_TO_TEST_DS_PROCESSED'])

In [16]:
df_train = pd.read_csv(PATH_TO_TRAIN_DS_RAW) 
df_test  = pd.read_csv(PATH_TO_TEST_DS_RAW) 
df_train.head()

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,1414,image_1415.jpeg,IF YOU WANT TO VIEW PARADISE,IF YOU WANT TO VIEW PARADISE SIMPLY LOOK AROU...,not_funny,not_sarcastic,not_offensive,motivational,positive
1,6459,image_6460.png,IF I HAD A BRICK FOR EVERY LIE HILLARY TOLD I ...,IF I HAD A BRICK FOR EVERY LIE HILLARY TOLD I ...,hilarious,general,very_offensive,motivational,positive
2,2302,image_2303.png,THAT THING OVER THERE CAN I EAT THAT quickmeme...,THAT THING OVER THERE CAN I EAT THAT quickmeme...,very_funny,general,not_offensive,motivational,very_positive
3,2416,image_2417.png,"MY DAD POINTED TO LIAM AND SAID ""WHEN DID DAVI...","MY DAD POINTED TO LIAM AND SAID ""WHEN DID DAVI...",not_funny,general,very_offensive,motivational,neutral
4,10,image_11.jpg,PROBABLY THE FIRST MAN TO DO 10 YEAR CHALLENGE...,PROBABLY THE FIRST MAN TO DO 10 YEAR CHALLENGE...,funny,general,very_offensive,motivational,negative


In [17]:
df_test.head()

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,5753,image_5754.png,"FLASHBACK: Trump said about Kim Jong-un: ""He s...","FLASHBACK: Trump said about Kim Jong-un: ""He s...",very_funny,twisted_meaning,not_offensive,motivational,positive
1,6597,image_6598.jpeg,My daughter wanted Cinderella-themed party,My daughter wanted Cinderella-themed party so...,very_funny,general,very_offensive,not_motivational,positive
2,5640,image_5641.jpg,HEY ROMNEY,HEY ROMNEY YOU MAD BRO?,funny,general,slight,motivational,neutral
3,5787,image_5788.jpg,GIFT HER LOLLYPOPS,GIFT HER LOLLYPOPS BECAUSE PRACTICE MAKES PER...,very_funny,general,not_offensive,not_motivational,positive
4,4785,image_4786.jpg,WHAT HITLER LOOKED LIKE WHEN THEY LOST THE WAR,WHAT HITLER LOOKED LIKE WHEN THEY LOST THE WAR,not_funny,not_sarcastic,not_offensive,not_motivational,negative


In [18]:
print(df_train.shape, df_test.shape)

(5593, 9) (1399, 9)


In [19]:
pd.merge(df_train, df_test, how='inner',  on=list(df_test.columns))

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment


In [20]:
df_train = df_train.drop_duplicates(subset=["text_ocr", "text_corrected", "humour", "sarcasm", "offensive", "motivational"], keep=False)
df_train.dropna(inplace = True)

df_test = df_test.drop_duplicates(subset=["text_ocr", "text_corrected", "humour", "sarcasm", "offensive", "motivational"], keep=False)
df_test.dropna(inplace = True)

df_train.head()

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,1414,image_1415.jpeg,IF YOU WANT TO VIEW PARADISE,IF YOU WANT TO VIEW PARADISE SIMPLY LOOK AROU...,not_funny,not_sarcastic,not_offensive,motivational,positive
1,6459,image_6460.png,IF I HAD A BRICK FOR EVERY LIE HILLARY TOLD I ...,IF I HAD A BRICK FOR EVERY LIE HILLARY TOLD I ...,hilarious,general,very_offensive,motivational,positive
2,2302,image_2303.png,THAT THING OVER THERE CAN I EAT THAT quickmeme...,THAT THING OVER THERE CAN I EAT THAT quickmeme...,very_funny,general,not_offensive,motivational,very_positive
3,2416,image_2417.png,"MY DAD POINTED TO LIAM AND SAID ""WHEN DID DAVI...","MY DAD POINTED TO LIAM AND SAID ""WHEN DID DAVI...",not_funny,general,very_offensive,motivational,neutral
4,10,image_11.jpg,PROBABLY THE FIRST MAN TO DO 10 YEAR CHALLENGE...,PROBABLY THE FIRST MAN TO DO 10 YEAR CHALLENGE...,funny,general,very_offensive,motivational,negative


In [21]:
class TextCleaner:

    stemmer = PorterStemmer()

    url_re_1 = r"\b(?:https?://|www\.)[a-z0-9-]+(\.[a-z0-9-]+)+(?:[/?].*)?" #remove a maioria das urls
    url_re_2 = r"(w{3}\.)*[a-zA-Z0-9]+\.{1}(co){1}[m]{0,1}\s{0,1}" # remove any.com urls
    url_re_3 = r"(w{3}\.)*[a-zA-Z0-9]+\.{1}(net){1}\s{0,1}" # remove any.net urls
    digits   = r'[0-9]' #remove digitos
   
    def clean(self, text):
        text = str(text)
        excess_whitespace_removed = ' '.join(text.split())
        s1 = re.sub(self.url_re_1, "", excess_whitespace_removed)
        s2 = re.sub(self.url_re_2, "", s1)
        s3 = re.sub(self.url_re_3, "", s2)
        s4 = re.sub(self.digits, "", s3)
        s4 = s4.translate(str.maketrans('', '', string.punctuation))
        
        return s4

    def stemmed(self, text):
      return ' '.join([self.stemmer.stem(t) for t in text.split()])

In [22]:
text_cleaner = TextCleaner()

In [23]:
df_train_cleaned = df_train.copy()[['image_name','text_corrected', 'humour',	'sarcasm',	'offensive',	'motivational', 'overall_sentiment']]
df_train_cleaned.loc[:,'text'] = df_train_cleaned.text_corrected\
                                .map(text_cleaner.clean)\
                                .map(text_cleaner.stemmed)                                
df_train_cleaned = df_train_cleaned.copy().drop('text_corrected', axis = 1).rename({'overall_sentiment':'target'}, axis = 1)
cols = df_train_cleaned.columns.tolist()
cols = cols[0:1] + cols[-1:] + cols[1:-1]
df_train_cleaned = df_train_cleaned[cols] 
df_train_cleaned.head()

,image_name,text,humour,sarcasm,offensive,motivational,target
0,image_1415.jpeg,if you want to view paradis simpli look around...,not_funny,not_sarcastic,not_offensive,motivational,positive
1,image_6460.png,if i had a brick for everi lie hillari told i ...,hilarious,general,very_offensive,motivational,positive
2,image_2303.png,that thing over there can i eat that,very_funny,general,not_offensive,motivational,very_positive
3,image_2417.png,my dad point to liam and said when did david b...,not_funny,general,very_offensive,motivational,neutral
4,image_11.jpg,probabl the first man to do year challeng chen...,funny,general,very_offensive,motivational,negative


In [24]:
df_test_cleaned = df_test.copy()[['image_name','text_corrected', 'humour',	'sarcasm',	'offensive',	'motivational', 'overall_sentiment']]
df_test_cleaned.loc[:,'text'] = df_test_cleaned.text_corrected\
                                .map(text_cleaner.clean)\
                                .map(text_cleaner.stemmed)                                
df_test_cleaned = df_test_cleaned.copy().drop('text_corrected', axis = 1).rename({'overall_sentiment':'target'}, axis = 1)
cols = df_test_cleaned.columns.tolist()
cols = cols[0:1] + cols[-1:] + cols[1:-1]
df_test_cleaned = df_test_cleaned[cols] 
df_test_cleaned.head()

,image_name,text,humour,sarcasm,offensive,motivational,target
0,image_5754.png,flashback trump said about kim jongun he speak...,very_funny,twisted_meaning,not_offensive,motivational,positive
1,image_6598.jpeg,my daughter want cinderellathem parti so i inv...,very_funny,general,very_offensive,not_motivational,positive
2,image_5641.jpg,hey romney you mad bro,funny,general,slight,motivational,neutral
3,image_5788.jpg,gift her lollypop becaus practic make perfect,very_funny,general,not_offensive,not_motivational,positive
4,image_4786.jpg,what hitler look like when they lost the war,not_funny,not_sarcastic,not_offensive,not_motivational,negative


In [25]:
df_train_cleaned.to_csv(PATH_TO_TRAIN_DS_PROCESSED, index=False)
df_test_cleaned.to_csv(PATH_TO_TEST_DS_PROCESSED, index=False)